# **PySpark Data cleaning and Neural Network**

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=5e47170264b97247a7523de75618e80f269d0252825ac44733fc602571969d8d
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder\
    .master("local[*]")\
    .appName("ML Term Deposit Pred")\
    .getOrCreate()
sc = spark.sparkContext

In [ ]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [ ]:
weather = spark.read.csv('/content/weather.csv', header = True, inferSchema = True)
weather.printSchema()
weather.show()

root
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- Evaporation: double (nullable = true)
 |-- Sunshine: string (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: string (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: string (nullable = true)
 |-- WindSpeed3pm: integer (nullable = true)
 |-- Humidity9am: integer (nullable = true)
 |-- Humidity3pm: integer (nullable = true)
 |-- Pressure9am: double (nullable = true)
 |-- Pressure3pm: double (nullable = true)
 |-- Cloud9am: integer (nullable = true)
 |-- Cloud3pm: integer (nullable = true)
 |-- Temp9am: double (nullable = true)
 |-- Temp3pm: double (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RISK_MM: double (nullable = true)
 |-- RainTomorrow: string (nullable = true)

+-------+-------+--------+-----------+--------+-----------+-------------+-

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

# Convert WindGustDir, WindDir9am, and WindDir3pm columns to numerical format
indexer = StringIndexer(inputCols=["WindGustDir", "WindDir9am", "WindDir3pm"],
                        outputCols=["WindGustDirIndex", "WindDir9amIndex", "WindDir3pmIndex"])

# Convert RainToday and RainTomorrow columns to numerical format
indexer_rain = StringIndexer(inputCols=["RainToday", "RainTomorrow"],
                             outputCols=["RainTodayIndex", "RainTomorrowIndex"])

# Apply the transformations
pipeline = Pipeline(stages=[indexer, indexer_rain])
weather = pipeline.fit(weather).transform(weather)

# Drop the original string columns
weather = weather.drop("WindGustDir", "WindDir9am", "WindDir3pm", "RainToday", "RainTomorrow")

In [ ]:
import pandas as pd
from pyspark.sql import functions as F

# Replace missing values in 'WindGustDir' column with 'NW'
weather = weather.withColumn('WindGustDirIndex', F.when(weather['WindGustDirIndex'].isNull(), 'NW').otherwise(weather['WindGustDirIndex']))

# Replace missing values in 'WindDir9am' column with 'SE'
weather = weather.withColumn('WindDir9amIndex', F.when(weather['WindDir9amIndex'].isNull(), 'SE').otherwise(weather['WindDir9amIndex']))

# Replace missing values in 'WindDir3pm' column with 'NW'
weather = weather.withColumn('WindDir3pmIndex', F.when(weather['WindDir3pmIndex'].isNull(), 'NW').otherwise(weather['WindDir3pmIndex']))

In [ ]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col

# Assuming 'weather' is your PySpark DataFrame

# Convert 'Sunshine' column to FloatType
weather = weather.withColumn('Sunshine', weather['Sunshine'].cast(FloatType()))
weather = weather.withColumn('WindGustSpeed', weather['WindGustSpeed'].cast(FloatType()))
weather = weather.withColumn('WindSpeed9am', weather['WindSpeed9am'].cast(FloatType()))

# Calculate median values
median_Sunshine = weather.approxQuantile('Sunshine', [0.5], 0.001)[0]
median_WindGustSpeed = weather.approxQuantile('WindGustSpeed', [0.5], 0.001)[0]
median_WindSpeed9am = weather.approxQuantile('WindSpeed9am', [0.5], 0.001)[0]

# Fill missing values with median values
weather = weather.na.fill({'Sunshine': median_Sunshine,
                           'WindGustSpeed': median_WindGustSpeed,
                           'WindSpeed9am': median_WindSpeed9am})

In [ ]:
weather.show()

+-------+-------+--------+-----------+--------+-------------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+-------+----------------+---------------+---------------+--------------+-----------------+
|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustSpeed|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RISK_MM|WindGustDirIndex|WindDir9amIndex|WindDir3pmIndex|RainTodayIndex|RainTomorrowIndex|
+-------+-------+--------+-----------+--------+-------------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+-------+----------------+---------------+---------------+--------------+-----------------+
|    8.0|   24.3|     0.0|        3.4|     6.3|         30.0|         6.0|          20|         68|         29|     1019.7|     1015.0|       7|       7|   14.4|   23.6|    3.6|             0.0|           14.0|          

In [ ]:
weather1 = spark.read.csv('/content/weather1.csv', header = True, inferSchema = True)
weather1.printSchema()
weather1.show()

root
 |-- Rainfall: double (nullable = true)
 |-- Cloud9am: integer (nullable = true)
 |-- RISK_MM: double (nullable = true)
 |-- RainTomorrow: string (nullable = true)

+--------+--------+-------+------------+
|Rainfall|Cloud9am|RISK_MM|RainTomorrow|
+--------+--------+-------+------------+
|     0.0|       7|    3.6|         Yes|
|     3.6|       5|    3.6|         Yes|
|     3.6|       8|   39.8|         Yes|
|    39.8|       2|    2.8|         Yes|
|     2.8|       7|    0.0|          No|
|     0.0|       7|    0.2|          No|
|     0.2|       4|    0.0|          No|
|     0.0|       6|    0.0|          No|
|     0.0|       7|   16.2|         Yes|
|    16.2|       7|    0.0|          No|
|     0.0|       1|    0.2|          No|
|     0.2|       0|    0.0|          No|
|     0.0|       0|    0.0|          No|
|     0.0|       1|    0.0|          No|
|     0.0|       0|    0.0|          No|
|     0.0|       0|    0.0|          No|
|     0.0|       7|    1.2|         Yes|
|     1.2|

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import when
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics


# Variables in dataset |Rainfall|Cloud9am|RISK_MM|RainTomorrow|

#Convert 'RainTomorrow' column to binary
weather1 = weather1.withColumn("RainTomorrow", when(weather1["RainTomorrow"] == "Yes", 1).otherwise(0))

weather1.show()

for col in weather1.columns[:-1]:
    weather1 = weather1.withColumn(col, weather1[col].cast(DoubleType()))

# Prepare the data for training
feature_columns = weather1.columns[:-1]  # last column is the target variable

vector_assembler = VectorAssembler(inputCols= feature_columns, outputCol="features")
data_preprocessed = vector_assembler.transform(weather1).select("features", "RainTomorrow")

# Split the data into training and testing sets
(training_data, test_data) = data_preprocessed.randomSplit([0.7, 0.3], seed=42)

# Define the neural network architecture
layers = [len(feature_columns), 2, 2, 2]  # Change this architecture according to your needs

# Create the Multilayer Perceptron model
mlp = MultilayerPerceptronClassifier(layers=layers, seed=42, labelCol="RainTomorrow")

# Train the model
model = mlp.fit(training_data)

# Make predictions
predictions = model.transform(test_data)

# Evaluate the model using Accuracy
evaluator = BinaryClassificationEvaluator(labelCol="RainTomorrow")
accuracy = evaluator.evaluate(predictions)

print("Test Accuracy =", accuracy)

# Evaluate the model using precision, recall and F1_score
evaluator2 = MulticlassClassificationEvaluator(labelCol="RainTomorrow", predictionCol="prediction",
                                              metricName="weightedPrecision")
precision = evaluator2.evaluate(predictions)

evaluator2.setMetricName("weightedRecall")
recall = evaluator2.evaluate(predictions)

evaluator2.setMetricName("f1")
f1_score = evaluator2.evaluate(predictions)

print("Precision =", precision)
print("Recall =", recall)
print("F1 Score =", f1_score)

+--------+--------+-------+------------+
|Rainfall|Cloud9am|RISK_MM|RainTomorrow|
+--------+--------+-------+------------+
|     0.0|       7|    3.6|           1|
|     3.6|       5|    3.6|           1|
|     3.6|       8|   39.8|           1|
|    39.8|       2|    2.8|           1|
|     2.8|       7|    0.0|           0|
|     0.0|       7|    0.2|           0|
|     0.2|       4|    0.0|           0|
|     0.0|       6|    0.0|           0|
|     0.0|       7|   16.2|           1|
|    16.2|       7|    0.0|           0|
|     0.0|       1|    0.2|           0|
|     0.2|       0|    0.0|           0|
|     0.0|       0|    0.0|           0|
|     0.0|       1|    0.0|           0|
|     0.0|       0|    0.0|           0|
|     0.0|       0|    0.0|           0|
|     0.0|       7|    1.2|           1|
|     1.2|       1|    0.6|           0|
|     0.6|       1|    0.0|           0|
|     0.0|       0|    0.0|           0|
+--------+--------+-------+------------+
only showing top

In [ ]:
spark.stop()